# Sunset Pier Hotels and Resorts - Data Wrangling

## 1. Introduction
This notebook contains the data wrangling step to solve the following problem:
```
How can Sunset Pier Hotels and Resorts implement data backed risk mitigation strategies for the next hotel season that
(a) reduce their loss in revenues due to cancellation to sub 10% and (b) do not dissuade the clients from booking,
therefore increasing overall revenues by 5%?
```

## 2. Project Links
* Dataset Source: https://www.kaggle.com/competitions/99-dapt-sao-ih-hotel-booking/data
* Project Proposal and Problem Statement Worksheet: http://localhost:8888/files/Capstone%202%20Project%20Proposal.pdf
* Github Repository: https://github.com/lojames/springboard-capstone-project-2

## 3. Imports

In [1]:
import pandas as pd
import numpy as np

## 4. Data Collection

Data sources:
* tb_hotel_traintest.csv - the provided training set
* tb_hotel_feat_valid_2.csv - the provided validation set

In [2]:
bookings_a = pd.read_csv('tb_hotel_traintest.csv')
bookings_b = pd.read_csv('tb_hotel_feat_valid_2.csv')

In [3]:
bookings_a.head(2)

,hotel,is_cancelled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date,id_booking
0,Resort Hotel,0,342,0,0,2,0.0,0,BB,PRT,...,NaN,NaN,0,Transient,0.0,0,0,2015-07-01,2015-07-01,0
1,Resort Hotel,0,737,0,0,2,0.0,0,BB,PRT,...,NaN,NaN,0,Transient,0.0,0,0,2015-07-01,2015-07-01,1


In [4]:
bookings_b.head(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date,id_booking
0,Resort Hotel,113,2,5,2,0.0,0,BB,NOR,Offline TA/TO,...,156.0,NaN,0,Transient-Party,82.88,0,2,2015-03-11,2015-07-02,47
1,Resort Hotel,5,1,0,2,0.0,0,BB,PRT,Online TA,...,240.0,NaN,0,Transient,97.00,0,0,2015-06-30,2015-07-05,132


In [5]:
set(bookings_a.columns)-set(bookings_b.columns)

{'is_cancelled'}

In [6]:
set(bookings_b.columns)-set(bookings_a.columns)

set()

As expected, the validation set does not have a is_cancelled column.  We will only use the data from set a to train and test models.

In [7]:
bookings = bookings_a

## 5. Data Definition

### 5.1. Column Names and Data Types

In [8]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113409 entries, 0 to 113408
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           113409 non-null  object 
 1   is_cancelled                    113409 non-null  int64  
 2   lead_time                       113409 non-null  int64  
 3   stays_in_weekend_nights         113409 non-null  int64  
 4   stays_in_week_nights            113409 non-null  int64  
 5   adults                          113409 non-null  int64  
 6   children                        113406 non-null  float64
 7   babies                          113409 non-null  int64  
 8   meal                            113409 non-null  object 
 9   country                         112951 non-null  object 
 10  market_segment                  113409 non-null  object 
 11  distribution_channel            113409 non-null  object 
 12  is_repeated_gues

In [9]:
# Helper code to create object to change data types
'''
sorted_bookings_cols = bookings.columns.sort_values()
temp = [print (f'    \'{b}\': ') for b in sorted_bookings_cols]
'''

"\nsorted_bookings_cols = bookings.columns.sort_values()\ntemp = [print (f'    '{b}': ') for b in sorted_bookings_cols]\n"

In [10]:
new_data_types = {
    'agent': 'category',
    'arrival_date': 'datetime64',
    'assigned_room_type': 'category',
    'company': 'category',
    'country': 'category',
    'customer_type': 'category',
    'deposit_type': 'category',
    'distribution_channel': 'category',
    'is_cancelled': 'category',
    'is_repeated_guest': 'category',
    'market_segment': 'category',
    'meal': 'category', 
    'reservation_status_date': 'datetime64',
    'reserved_room_type': 'category',
    'hotel': 'string'
}

bookings = bookings.astype(new_data_types)
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113409 entries, 0 to 113408
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           113409 non-null  string        
 1   is_cancelled                    113409 non-null  category      
 2   lead_time                       113409 non-null  int64         
 3   stays_in_weekend_nights         113409 non-null  int64         
 4   stays_in_week_nights            113409 non-null  int64         
 5   adults                          113409 non-null  int64         
 6   children                        113406 non-null  float64       
 7   babies                          113409 non-null  int64         
 8   meal                            113409 non-null  category      
 9   country                         112951 non-null  category      
 10  market_segment                  113409 non-null  categor

### 5.2. Column Descriptions

In [11]:
description = bookings.describe()
description.T

,count,mean,std,min,25%,50%,75%,max
lead_time,113409.0,104.109074,106.894825,0.00,18.0,69.0,161.0,737.0
stays_in_weekend_nights,113409.0,0.927907,0.998723,0.00,0.0,1.0,2.0,19.0
stays_in_week_nights,113409.0,2.500498,1.906670,0.00,1.0,2.0,3.0,50.0
adults,113409.0,1.857304,0.583753,0.00,2.0,2.0,2.0,55.0
children,113406.0,0.104227,0.398976,0.00,0.0,0.0,0.0,10.0
babies,113409.0,0.007980,0.098027,0.00,0.0,0.0,0.0,10.0
previous_cancellations,113409.0,0.087101,0.844538,0.00,0.0,0.0,0.0,26.0
previous_bookings_not_canceled,113409.0,0.136330,1.497662,0.00,0.0,0.0,0.0,72.0
booking_changes,113409.0,0.220917,0.649771,0.00,0.0,0.0,0.0,21.0
days_in_waiting_list,113409.0,2.326200,17.613897,0.00,0.0,0.0,0.0,391.0


### 5.3. Counts and Percents of Unique Values

In [12]:
num_unique = bookings.nunique()
print (f'Number of Unique Values Per Column\n{num_unique}')

Number of Unique Values Per Column
hotel                                  2
is_cancelled                           2
lead_time                            478
stays_in_weekend_nights               17
stays_in_week_nights                  35
adults                                14
children                               5
babies                                 5
meal                                   5
country                              174
market_segment                         8
distribution_channel                   5
is_repeated_guest                      2
previous_cancellations                15
previous_bookings_not_canceled        72
reserved_room_type                    10
assigned_room_type                    12
booking_changes                       20
deposit_type                           3
agent                                327
company                              348
days_in_waiting_list                 127
customer_type                          4
adr                   

In [13]:
percent_unique = num_unique/len(bookings)*100
print (f'Percent Unique Values By Column\n{percent_unique}')

Percent Unique Values By Column
hotel                               0.001764
is_cancelled                        0.001764
lead_time                           0.421483
stays_in_weekend_nights             0.014990
stays_in_week_nights                0.030862
adults                              0.012345
children                            0.004409
babies                              0.004409
meal                                0.004409
country                             0.153427
market_segment                      0.007054
distribution_channel                0.004409
is_repeated_guest                   0.001764
previous_cancellations              0.013226
previous_bookings_not_canceled      0.063487
reserved_room_type                  0.008818
assigned_room_type                  0.010581
booking_changes                     0.017635
deposit_type                        0.002645
agent                               0.288337
company                             0.306854
days_in_waiting_list   

### 5.4. Range of Values

In [14]:
description.loc[['min','max']].T

,min,max
lead_time,0.00,737.0
stays_in_weekend_nights,0.00,19.0
stays_in_week_nights,0.00,50.0
adults,0.00,55.0
children,0.00,10.0
babies,0.00,10.0
previous_cancellations,0.00,26.0
previous_bookings_not_canceled,0.00,72.0
booking_changes,0.00,21.0
days_in_waiting_list,0.00,391.0


### 5.5. Considerations

***Do your column names correspond to what those columns store?***

Yes.  A quick examination done in 4 of the first 20 rows seems to indicate so.  Further examination will be done later in the data cleaning step.

***Check the data types of your columns. Are they sensible?***

Yes.  Datatypes were changed accordingly above in 5.1.

***Calculate summary statistics for each of your columns, such
as mean, median, mode, standard deviation, range, and
number of unique values. What does this tell you about your
data?***

These summary statistics reveal the central tendency, dispersion, and shape of the dataset's distributions.  Furthermore, the validity of the data can be analyzed through these statistics.

***What do you now need to investigate?***

Missing values, duplicates, and outliers.

## 6. Data Cleaning

In [15]:
bookings_cleaned = bookings.copy(deep=True)

### 6.1. Missing Values

In [16]:
bookings_cleaned.isna().sum()

hotel                                  0
is_cancelled                           0
lead_time                              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               3
babies                                 0
meal                                   0
country                              458
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              15491
company                           106972
days_in_waiting_list                   0
customer_type                          0
adr                                    0
required_car_par

The following columns have missing values that need to be addressed:
* `children`
* `country`
* `agent`
* `company`

#### 6.1.1 'children' Missing Values
Only 3 reservations have missing values in the children column.

In [17]:
bookings_cleaned['children'].value_counts()

0.0     105210
1.0       4652
2.0       3471
3.0         72
10.0         1
Name: children, dtype: int64

The mode of the column is 0. Since the vast majority of reservations have 0 children (more than 92.77%), imputing with 0 is the best quick and dirty approach.

In [18]:
bookings_cleaned['children'].fillna(0, inplace=True)
bookings_cleaned['children'].value_counts()

0.0     105213
1.0       4652
2.0       3471
3.0         72
10.0         1
Name: children, dtype: int64

#### 6.1.2 'country' Missing Values
There are only 458 missing values for 'country'.  While this number is well under the threshold for dropping the rows with the missing values, the missing values can be assigned to an unused 3 letter code.

In [19]:
bookings_cleaned['country'].value_counts()

PRT    46213
GBR    11487
FRA     9890
ESP     8162
DEU     6919
       ...  
MDG        1
MMR        1
SMR        1
MRT        1
PYF        1
Name: country, Length: 174, dtype: int64

While the documentation specifies that the coutnry values are of the ISO 3155–3:2013 standard, this standard is not readily viewable online. The ISO 3166-1 alpha 3 code standard will be referenced (https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3) instead. The codes have been saved to the text file `country_codes.txt`.

A quick sanity check:

In [20]:
# Creating a dictionary of countries with ISO 3166-1 alpha 3 as the key and the country name as value.
country_codes = []
country_names = []
with open('country_codes.txt', encoding = 'utf8') as file:
    for line in file:
        country_codes += [line[0:3]]
        country_names += [line[5:-1]]
countries_dictionary = dict(zip(country_codes,country_names))
countries_dictionary['PRT']

'Portugal'

In [21]:
country_codes_set = set(country_codes)
bookings[~bookings['country'].isin(country_codes_set)]['country'].unique()

[NaN, 'CN', 'TMP']
Categories (174, object): ['ABW', 'AGO', 'AIA', 'ALB', ..., 'VGB', 'VNM', 'ZAF', 'ZWE']

Unfortunately, it's clear that the standards are not exactly the same; however the standards seem to be similar.  Again since the ISO 3155–3:2013 standard is not readily available, we have to do with what we have.

With regards to missing values, the ISO 3166-1 alpha 3 standards has various 3 letter codes that can be user assigned including `ZZZ`.  `ZZZ` will be used to specify missing values.

In [22]:
bookings_cleaned['country'] = bookings_cleaned['country'].cat.add_categories('ZZZ')
bookings_cleaned['country'].fillna('ZZZ', inplace=True)
bookings_cleaned['country'].isna().sum()

0

In [23]:
bookings_cleaned['country'].value_counts()['ZZZ']

458

#### 6.1.3 'agent' Missing Values
There are 15491 missing

In [24]:
print (f'{15491/len(bookings)*100:.2f}% of values are missing.')

13.66% of values are missing.


While we can opt to drop the reservations with missing agent values, it may be better to assign the missing values to an unused value.

In [50]:
np.array(bookings['agent'].cat.categories).min()

1.0

Since the lowest value used for the agent category is 1, we can safetly designate 0 as missing values.

In [51]:
bookings_cleaned['agent'] = bookings_cleaned['agent'].cat.add_categories(0)

In [52]:
bookings_cleaned['agent'].fillna(0, inplace=True)
bookings_cleaned['agent'].isna().sum()

0

In [59]:
bookings_cleaned['agent'].value_counts()[0]

15491

#### 6.1.4 'company' Missing Values
There are 106972 missing values for company.

In [54]:
print (f'{106972/len(bookings)*100:.2f}% of values are missing.')

94.32% of values are missing.


Since the vast majority of rows are missing values for `company`, it's best if missing values are reassigned.

In [55]:
np.array(bookings['company'].cat.categories).min()

6.0

0 is a safe value to reassign.

In [56]:
bookings_cleaned['company'] = bookings_cleaned['company'].cat.add_categories(0)

In [57]:
bookings_cleaned['company'].fillna(0, inplace=True)
bookings_cleaned['company'].isna().sum()

0

In [60]:
bookings_cleaned['company'].value_counts()[0]

106972

### 6.2 Duplicated Values

In [73]:
bookings[bookings.duplicated()]

,hotel,is_cancelled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date,id_booking


Note that the dataset is curated.

In [68]:
len(bookings['id_booking'].unique())==len(bookings)

True

In the case of this dataset, faith must be placed that the id_booking has been correctly generated for each unique booking.  Without this column, it's virtually impossible to definitively tell if certain bookings are erroneous duplicates of each other.  More specifically, separate bookings without the `id_booking` column can have identical column values for the remaining columns and still refer to different reservations -- these is no sure method of knowing.  For example, 4 different couples may have 1 individual make reservations for all the couples all originating from the same country with the same itinerary.

**Due to this fact and the fact that this is a curated dataset, no rows will be culled from this dataset.**